In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

label.txt 을 읽어서 파싱 > x, y 를 준비

In [2]:
folder = 'C:/Users/user/Downloads/ocr/'
path= folder +'label.txt'
dst_size = (32, 32) #w,h
CHARACTER = 'AB가' # 모든 글자들

In [3]:
df = pd.read_csv(path, encoding='cp949')

In [4]:
print(df.head(1))

  class   file
0     A  A.png


이미지를 읽이서 리사이즈하고 배열로 변환하자

In [5]:
image_arr_list = []
for image_path in df['file']:
    image = Image.open(folder+image_path)
    image_resized = image.resize(dst_size)
    image_arr = np.array(image_resized)
    image_arr_list.append(image_arr)
print(len(image_arr_list))

4


In [6]:
y_list = []
for v in df['class'].values:    
    #print(v, CHARACTER.index(v))
    y_list.append(CHARACTER.index(v))
print(len(y_list))

4


In [7]:
x = np.stack(image_arr_list, 0)
y = np.stack(y_list, 0)
print(x.shape, y.shape, x.dtype, np.max(x))

(4, 32, 32, 3) (4,) uint8 255


Y = CNN(X)
모양 (갯수, h, w, c)
X (m, 32, 32, 3)
conv1 : (m, 32, 32, 32)
pool  : (m, 16, 16, 32)
conv2 : (m, 16, 16, 64)
pool  : (m,  8,  8, 64)
conv3 : (m,  8,  8, 128)
pool  : (m,  4,  4, 128)
conv4 : (m,  4,  4, 128)
pool  : (m,  2,  2, 128)
conv5 : (m,  2,  2, 128)
pool  : (m,  1,  1, 128)
squeeze (m,  128)
fc    : (m,  k)

Y (m)
Y_one_hot (m, k)

# 숫자 샘플로 테스트
from sklearn.datasets import load_digits
x, y = load_digits(return_X_y=True)
x.shape, y.shape
x = np.reshape(x, [-1,8,8,1])
print(x.shape)

In [8]:
import keras
from keras.models import Sequential
k = len(CHARACTER)
#k = 10 #숫자 샘플
#dst_size = (8,8)

Using TensorFlow backend.


In [14]:
# 케라스 
#tf.reset_default_graph() #초기화
model = Sequential()
model.add(keras.layers.Conv2D(32,[3,3], padding='same', 
                              input_shape=(dst_size[1], dst_size[0], 3)))
model.add(keras.layers.MaxPool2D(pool_size=[2,2] ))
model.add(keras.layers.Conv2D(64,[3,3], padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=[2,2]))
model.add(keras.layers.Conv2D(128,[3,3], padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=[2,2]))
model.add(keras.layers.Conv2D(128,[3,3], padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=[2,2]))
model.add(keras.layers.Conv2D(128,[3,3], padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=[2,2]))
model.add(keras.layers.Conv2D(128,[3,3], padding='same'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(k))
model.add(keras.layers.Activation('softmax'))

opt = keras.optimizers.rmsprop(lr=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', 
              metrics=['accuracy'])
one_hot_labels = keras.utils.to_categorical(y, num_classes=k)

In [15]:
xf = x.astype(np.float32)/255
model.fit(xf, one_hot_labels, epochs=20, verbose=2)

Epoch 1/20
 - 1s - loss: 1.0959 - acc: 0.0000e+00
Epoch 2/20
 - 0s - loss: 2.5223 - acc: 0.5000
Epoch 3/20
 - 0s - loss: 1.1603 - acc: 0.2500
Epoch 4/20
 - 0s - loss: 1.2456 - acc: 0.5000
Epoch 5/20
 - 0s - loss: 1.0773 - acc: 0.5000
Epoch 6/20
 - 0s - loss: 0.9254 - acc: 1.0000
Epoch 7/20
 - 0s - loss: 0.7833 - acc: 0.5000
Epoch 8/20
 - 0s - loss: 0.6149 - acc: 1.0000
Epoch 9/20
 - 0s - loss: 0.4387 - acc: 1.0000
Epoch 10/20
 - 0s - loss: 0.5763 - acc: 0.5000
Epoch 11/20
 - 0s - loss: 1.1090 - acc: 0.2500
Epoch 12/20
 - 0s - loss: 0.4133 - acc: 0.7500
Epoch 13/20
 - 0s - loss: 0.1397 - acc: 1.0000
Epoch 14/20
 - 0s - loss: 0.0606 - acc: 1.0000
Epoch 15/20
 - 0s - loss: 0.0344 - acc: 1.0000
Epoch 16/20
 - 0s - loss: 0.0226 - acc: 1.0000
Epoch 17/20
 - 0s - loss: 0.0162 - acc: 1.0000
Epoch 18/20
 - 0s - loss: 0.0124 - acc: 1.0000
Epoch 19/20
 - 0s - loss: 0.0097 - acc: 1.0000
Epoch 20/20
 - 0s - loss: 0.0078 - acc: 1.0000


In [11]:
score = model.evaluate(x, one_hot_labels)
print('loss', score[0], 'score', score[1])

4/4 [==============================] - 0s 18ms/step
loss 8.05904769897461 score 0.5


# 텐서플로우

#import tensorflow.contrib.slim as slim
tf.reset_default_graph() #초기화
def model(x):
    x = tf.cast(x, tf.float32) # uint8 > float32 데이터 타입 변환
    x = x / 255 # [0,255] > [0, 1] 정규화 
    net = slim.conv2d(x, 32, [3,3], scope='conv1')
    net = slim.max_pool2d(net, 2)
    net = slim.conv2d(net, 64, [3,3], scope='conv2')
    net = slim.max_pool2d(net, 2)
    net = slim.conv2d(net, 128, [3,3], scope='conv3')
    net = slim.max_pool2d(net, 2)
    net = slim.conv2d(net, 128, [3,3], scope='conv4')
    net = slim.max_pool2d(net, 2)
    net = slim.conv2d(net, 128, [3,3], scope='conv5')
    net = slim.max_pool2d(net, 2)
    return net

X = tf.placeholder(tf.uint8, [None, dst_size[1], dst_size[0], 3])
y = tf.placeholder(tf.int64, [None])
h = model(X)
print(h)